# 音声ファイルからSRT字幕ファイルを生成

このノートブックは、音声ファイル（MP3など）を日本語音声認識し、タイムライン付きのSRT字幕ファイルを自動生成するツールです。Whisper AIを使用して高精度な文字起こしを行います。

**主な機能:**
- 基本的な音声認識とSRT生成
- 長時間音声のセグメント分割処理
- 単語レベルのタイムスタンプ最適化

In [ ]:
!pip install -U openai-whisper 
!pip install numpy
!pip install pydub
!sudo apt-get install -y ffmpeg
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
#音声ファイルだけの場合は音声認識を行い、タイムライン付きのSRTファイルを生成します。ffmpegのインストールが必須です。
#主に日本語能力試験の音声で使用します。
#https://www.gyan.dev/ffmpeg/builds/

import whisper
import torch
import os

AUDIO_PATH = "/path/to/your/audio.mp3"        # 元の音声ファイルのパス

def transcribe_audio(audio_path):
    print("🔍 音声を認識しています...")
    
    # 必ずGPUを使用する
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用デバイス: {device}")
    
    # モデルを読み込む
    model = whisper.load_model("medium", device=device)
    
    # 音声を文字起こしする
    result = model.transcribe(audio_path, language='ja')
    
    # 現在のスクリプトディレクトリを取得
    script_dir = os.path.dirname(os.path.abspath(__file__))
    output_path = os.path.join(script_dir, "output.srt")
    
    # SRTファイルを保存する
    with open(output_path, "w", encoding="utf-8") as f:
        for i, segment in enumerate(result["segments"]):
            start = segment["start"]
            end = segment["end"]
            text = segment["text"].strip()
            f.write(f"{i+1}\n")
            f.write(f"{format_time(start)} --> {format_time(end)}\n")
            f.write(f"{text}\n\n")
    
    print(f"✅ 文字起こしが完了しました！")
    print(f"📁 SRT ファイルの保存場所: {output_path}")
    print(f"📝 合計 {len(result['segments'])} 個の字幕セグメントを生成しました")

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    millis = int((secs - int(secs)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(secs):02},{millis:03}"

if __name__ == "__main__":
    try:
        transcribe_audio(AUDIO_PATH)
    except FileNotFoundError:
        print(f"❌ 音声ファイルが見つかりません: {AUDIO_PATH}")
    except Exception as e:
        print(f"❌ 文字起こし中にエラーが発生しました: {e}")


In [ ]:
import whisper
import torch
import os
import math
from pydub import AudioSegment
from typing import List, Dict
import tempfile
import shutil

AUDIO_PATH = "/path/to/your/long_audio.mp3"        # 元の音声ファイルのパス
SEGMENT_DURATION = 5* 60  # 5分（300秒）
OVERLAP_DURATION = 5        # 5秒のオーバーラップで途切れを防ぐ
OUTPUT_DIR = None           # 出力ディレクトリ。Noneなら自動で決定

class SegmentedTranscriber:
    def __init__(self, model_size="medium", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"🖥️  使用デバイス: {self.device}")
        
        # モデルを読み込む
        print("📦 Whisperモデルを読み込んでいます...")
        self.model = whisper.load_model(model_size, device=self.device)
        print("✅ モデルの読み込みが完了しました")

    def split_audio(self, audio_path: str) -> List[Dict]:
        """音声を10分ごとのセクションに分割する"""
        print("🔪 音声を分割しています...")
        
        # 音声を読み込む
        audio = AudioSegment.from_file(audio_path)
        total_duration = len(audio) / 1000  # 秒単位に変換する
        
        print(f"📊 音声の総時間: {self.format_time(total_duration)}")
        
        segments = []
        segment_count = math.ceil(total_duration / SEGMENT_DURATION)
        
        # 一時ディレクトリを作成
        temp_dir = tempfile.mkdtemp()
        
        for i in range(segment_count):
            start_time = i * SEGMENT_DURATION
            
            # 終了時刻を計算し、最後の区間以外はオーバーラップを加える
            if i < segment_count - 1:
                end_time = min(start_time + SEGMENT_DURATION + OVERLAP_DURATION, total_duration)
            else:
                end_time = total_duration
            
            # 音声セグメントを抽出
            start_ms = int(start_time * 1000)
            end_ms = int(end_time * 1000)
            segment_audio = audio[start_ms:end_ms]
            
            # 一時ファイルを保存
            temp_file = os.path.join(temp_dir, f"segment_{i:03d}.wav")
            segment_audio.export(temp_file, format="wav")
            
            segments.append({
                'file_path': temp_file,
                'start_offset': start_time,
                'end_offset': end_time,
                'segment_index': i,
                'actual_duration': (end_ms - start_ms) / 1000
            })
            
            print(f"📋 セグメント {i+1}/{segment_count}: {self.format_time(start_time)} - {self.format_time(end_time)}")
        
        return segments, temp_dir

    def transcribe_segment(self, segment_info: Dict) -> List[Dict]:
        """単一の音声セグメントを文字起こし"""
        file_path = segment_info['file_path']
        start_offset = segment_info['start_offset']
        segment_index = segment_info['segment_index']
        
        print(f"🎯 セグメント {segment_index + 1} を文字起こししています...")
        
        # 音声区間を文字起こしする
        result = self.model.transcribe(file_path, language='ja')
        
        # タイムスタンプを調整
        adjusted_segments = []
        for seg in result["segments"]:
            # タイムスタンプにオフセットを加える
            adjusted_start = seg["start"] + start_offset
            adjusted_end = seg["end"] + start_offset
            
            adjusted_segments.append({
                'start': adjusted_start,
                'end': adjusted_end,
                'text': seg["text"].strip(),
                'segment_index': segment_index
            })
        
        return adjusted_segments

    def merge_overlapping_segments(self, all_segments: List[List[Dict]]) -> List[Dict]:
        """重複区間を統合して重複内容を削除"""
        print("🔗 セグメントを結合し、重複部分を処理しています...")
        
        merged_segments = []
        
        for i, segments in enumerate(all_segments):
            if i == 0:
                # 最初の区間はそのまま追加
                merged_segments.extend(segments)
            else:
                # 次以降の区間は重複を処理する
                overlap_start = i * SEGMENT_DURATION
                
                # 重複区間以降の部分のみ追加
                for seg in segments:
                    if seg['start'] >= overlap_start:
                        merged_segments.append(seg)
        
        # 時間順に並び替える
        merged_segments.sort(key=lambda x: x['start'])
        
        return merged_segments

    def save_srt(self, segments: List[Dict], output_path: str):
        """SRTファイルを保存する"""
        print("💾 SRT ファイルを保存しています...")
        
        with open(output_path, "w", encoding="utf-8") as f:
            for i, segment in enumerate(segments):
                start = segment["start"]
                end = segment["end"]
                text = segment["text"].strip()
                
                if text:  # 空でないテキストだけを保存
                    f.write(f"{i+1}\n")
                    f.write(f"{self.format_time(start)} --> {self.format_time(end)}\n")
                    f.write(f"{text}\n\n")
        
        print(f"✅ SRT ファイルの保存が完了しました: {output_path}")
        print(f"📝 合計 {len([s for s in segments if s['text'].strip()])} 個の字幕セグメントを生成しました")

    def transcribe_audio(self, audio_path: str):
        """メインの文字起こし関数"""
        print("🚀 セグメント分けした文字起こしを開始します...")
        
        # 音声を分割
        segments_info, temp_dir = self.split_audio(audio_path)
        
        try:
            all_segments = []
            
            # 区間ごとに文字起こし
            for segment_info in segments_info:
                segment_result = self.transcribe_segment(segment_info)
                all_segments.append(segment_result)
                
                # GPUメモリを解放
                if self.device == "cuda":
                    torch.cuda.empty_cache()
            
            # 重複区間を統合する
            final_segments = self.merge_overlapping_segments(all_segments)
            
            # 出力ファイルのパスを作成
            if OUTPUT_DIR:
                script_dir = OUTPUT_DIR
            else:
                try:
                    # スクリプトディレクトリの取得を試みる
                    script_dir = os.path.dirname(os.path.abspath(__file__))
                except NameError:
                    # Jupyter/IPython では現在の作業ディレクトリを使用
                    script_dir = os.getcwd()
            
            base_name = os.path.splitext(os.path.basename(audio_path))[0]
            output_path = os.path.join(script_dir, f"{base_name}_segmented.srt")
            
            # SRTファイルを保存する
            self.save_srt(final_segments, output_path)
            
            # 統計情報を表示
            total_duration = max(seg['end'] for seg in final_segments) if final_segments else 0
            print(f"\n📊 文字起こし統計:")
            print(f"   • 総時間: {self.format_time(total_duration)}")
            print(f"   • セグメント数: {len(segments_info)}")
            print(f"   • 字幕数: {len(final_segments)}")
            print(f"   • 出力ファイル: {output_path}")
            
        finally:
            # 一時ファイルを削除
            print("🧹 一時ファイルを削除しています...")
            shutil.rmtree(temp_dir, ignore_errors=True)

    @staticmethod
    def format_time(seconds):
        """SRT形式のタイムスタンプに整形"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        millis = int((secs - int(secs)) * 1000)
        return f"{hours:02}:{minutes:02}:{int(secs):02},{millis:03}"

def main():
    """メイン関数"""
    try:
        # 音声ファイルの有無を確認
        if not os.path.exists(AUDIO_PATH):
            print(f"❌ 音声ファイルが見つかりません: {AUDIO_PATH}")
            return
        
        # 依存関係を確認
        try:
            from pydub import AudioSegment
        except ImportError:
            print("❌ 依存パッケージ pydub が不足しています。インストールしてください:")
            print("   pip install pydub")
            return
        
        # 文字起こし用クラスを作成
        transcriber = SegmentedTranscriber(model_size="medium")
        
        # 文字起こしを開始
        transcriber.transcribe_audio(AUDIO_PATH)
        
        print("\n🎉 セグメント分けした文字起こしが完了しました！")
        
    except Exception as e:
        print(f"❌ 文字起こし中にエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


In [ ]:
import whisper
import torch
import numpy as np
from pathlib import Path

AUDIO_PATH = "/path/to/your/audio.mp3"        # 元の音声ファイルのパス

def transcribe_audio_optimized(audio_path):
    print("🔍 音声を認識しています...")
    
    # 必ずGPUを使用する
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用デバイス: {device}")
    
    # モデルをGPUに読み込み、large-v2を推奨（時間軸の精度が向上）
    model = whisper.load_model("large-v2", device=device)
    
    # パラメータを最適化して時間軸の精度を高める
    result = model.transcribe(
        audio_path, 
        language='ja',
        
        # 重要パラメータ：単語レベルのタイムスタンプ
        word_timestamps=True,
        
        # デコード品質を上げるパラメータ
        beam_size=5,                    # ビームサーチで精度を向上
        best_of=5,                      # 複数回サンプリングして最良結果を採用
        temperature=0.0,                # 決定的な出力でランダム性を避ける
        
        # 文脈の累積誤差を避ける
        condition_on_previous_text=False,
        
        # 無音と低品質を検出
        no_speech_threshold=0.6,        # 無音検出の閾値
        logprob_threshold=-1.0,         # 低信頼度のフィルタ
        compression_ratio_threshold=2.4, # 重複コンテンツのフィルタ
        
        # 日本語向けの最適化
        initial_prompt="以下は日本語の音声です。",
        
        # VADパラメータの最適化
        prepend_punctuations='"\'¿([{-',
        append_punctuations='"\'.。,，!！?？:：")]}、'
    )
    
    print(f"✅ 認識完了、合計 {len(result['segments'])} 個のセグメント")
    
    # 単語タイムスタンプで区間境界を最適化
    optimized_segments = optimize_segment_boundaries(result['segments'])
    
    # SRTファイルを生成
    output_path = "output_optimized.srt"
    save_srt_with_optimized_timing(optimized_segments, output_path)
    
    return result

def optimize_segment_boundaries(segments):
    """
    単語レベルのタイムスタンプに基づいてセグメントの境界を最適化し、中間時間の偏差を減らす
    """
    optimized_segments = []
    
    for segment in segments:
        # 単語タイムスタンプがあればより正確な境界を使う
        if 'words' in segment and segment['words']:
            words = segment['words']
            
            # 最初と最後の単語のタイムスタンプを境界として使う
            first_word_start = words[0]['start']
            last_word_end = words[-1]['end']
            
            # 最適化済みの区間を作成
            optimized_segment = {
                'id': segment['id'],
                'start': first_word_start,
                'end': last_word_end,
                'text': segment['text'],
                'words': words
            }
            
            # 異常なタイムスタンプを検査して修正
            if optimized_segment['start'] >= optimized_segment['end']:
                optimized_segment['start'] = segment['start']
                optimized_segment['end'] = segment['end']
                
        else:
            # 単語タイムスタンプが無い場合は元の値を利用
            optimized_segment = segment
            
        optimized_segments.append(optimized_segment)
    
    # 区間の時間が重ならないようにする
    optimized_segments = fix_overlapping_segments(optimized_segments)
    
    return optimized_segments

def fix_overlapping_segments(segments):
    """
    重複するセグメントのタイムスタンプを修正
    """
    if len(segments) <= 1:
        return segments
    
    fixed_segments = [segments[0]]
    
    for i in range(1, len(segments)):
        current = segments[i].copy()
        previous = fixed_segments[-1]
        
        # 現在の開始時刻が前の終了時刻より早い場合
        if current['start'] < previous['end']:
            # 境界を調整：中間点で分割する
            mid_point = (previous['end'] + current['start']) / 2
            previous['end'] = mid_point
            current['start'] = mid_point
            
            # 前の区間を更新
            fixed_segments[-1] = previous
        
        fixed_segments.append(current)
    
    return fixed_segments

def save_srt_with_optimized_timing(segments, output_path):
    """
    最適化されたタイムスタンプのSRTファイルを保存
    """
    with open(output_path, "w", encoding="utf-8") as f:
        for i, segment in enumerate(segments):
            start = segment["start"]
            end = segment["end"]
            text = segment["text"].strip()
            
            f.write(f"{i+1}\n")
            f.write(f"{format_time(start)} --> {format_time(end)}\n")
            f.write(f"{text}\n\n")
    
    print(f"📁 最適化されたSRTファイルを保存しました: {output_path}")

def format_time(seconds):
    """
    タイムスタンプをフォーマット、精度を向上
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    millis = int((secs - int(secs)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(secs):02},{millis:03}"

def process_long_audio_with_overlap(audio_path, segment_length=600, overlap=60):
    """
    45-60分の長い音声に対するセグメント処理で、累積誤差を減らす
    """
    print("🎵 長い音声ファイルを処理しています...")
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("large-v2", device=device)
    
    # 音声を読み込んで総再生時間を取得
    audio = whisper.load_audio(audio_path)
    total_duration = len(audio) / whisper.audio.SAMPLE_RATE
    
    print(f"音声の総時間: {total_duration/60:.1f} 分")
    
    all_segments = []
    current_time = 0
    segment_id = 0
    
    while current_time < total_duration:
        start_sample = int(current_time * whisper.audio.SAMPLE_RATE)
        end_time = min(current_time + segment_length, total_duration)
        end_sample = int(end_time * whisper.audio.SAMPLE_RATE)
        
        # 音声クリップを抽出
        audio_segment = audio[start_sample:end_sample]
        
        print(f"セグメントを処理中: {current_time/60:.1f} - {end_time/60:.1f} 分")
        
        # 同じ最適化パラメータで区間を文字起こし
        result = model.transcribe(
            audio_segment,
            language='ja',
            word_timestamps=True,
            beam_size=5,
            best_of=5,
            temperature=0.0,
            condition_on_previous_text=False,
            no_speech_threshold=0.6,
            logprob_threshold=-1.0,
            compression_ratio_threshold=2.4,
            initial_prompt="以下は日本語の音声です。"
        )
        
        # タイムスタンプを調整し全体結果に追加
        for segment in result['segments']:
            segment['start'] += current_time
            segment['end'] += current_time
            segment['id'] = segment_id
            segment_id += 1
            
            # 単語タイムスタンプを調整
            if 'words' in segment:
                for word in segment['words']:
                    word['start'] += current_time
                    word['end'] += current_time
            
            all_segments.append(segment)
        
        current_time += segment_length - overlap
    
    # 全体のタイムスタンプを最適化
    optimized_segments = optimize_segment_boundaries(all_segments)
    
    # 結果を保存
    output_path = "output_long_optimized.srt"
    save_srt_with_optimized_timing(optimized_segments, output_path)
    
    print(f"✅ 長い音声の処理が完了しました、合計 {len(optimized_segments)} 個のセグメント")
    return optimized_segments

# メイン関数
def main():
    print("🎌 日本語 Whisper タイムライン最適化文字起こし")
    
    # 音声の長さを確認して処理方式を決める
    try:
        audio = whisper.load_audio(AUDIO_PATH)
        duration_minutes = len(audio) / whisper.audio.SAMPLE_RATE / 60
        
        print(f"検出された音声の長さ: {duration_minutes:.1f} 分")
        
        if duration_minutes > 30:
            print("長い音声最適化モードを使用します...")
            process_long_audio_with_overlap(AUDIO_PATH)
        else:
            print("標準最適化モードを使用します...")
            transcribe_audio_optimized(AUDIO_PATH)
            
    except Exception as e:
        print(f"❌ 処理に失敗しました: {e}")
        print("標準モードを試します...")
        transcribe_audio_optimized(AUDIO_PATH)

if __name__ == "__main__":
    main()
